# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
# json.load((open('data/world_bank_projects_less.json'))) # comment out for better visibility


In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

### Question 1

1. Find the 10 countries with most projects


In [8]:
# load json file into 'json_all'
json_all = pd.read_json('data/world_bank_projects.json')

## Data Exploration 
col = json_all.columns
num_rows = len(json_all)

# print(col)         # explore the column names
# print(num_rows)    # 500 rows of data

# Make new dataframe with columns of our interest and groupby 'countryname' column 
# aggfunc = .count()

new_df = json_all[['countrycode','country_namecode','countryname','countryshortname']]
grouped = new_df.groupby('countryname').count()

# check to see all the columns have the same information across rows by comparing the values in each column across rows
check_equality = (grouped['countrycode'] == grouped['country_namecode']).all()
check_equality_2 = (grouped['countryshortname'] == grouped['country_namecode']).all()

# print statement for readability 
print("Are all the values between 'countrycode' column and 'country_namecode' column same? {}\n\
Are all the values between 'countryshortname' column and 'country_namecode' column same? {}" \
.format(check_equality, check_equality_2))

# Answer (top 10 countries with most projects)
sorted_df = grouped.sort_values(by=['countrycode'], ascending=False)
sorted_df.head(10)



Are all the values between 'countrycode' column and 'country_namecode' column same? True
Are all the values between 'countryshortname' column and 'country_namecode' column same? True


,countrycode,country_namecode,countryshortname
countryname,,,
People's Republic of China,19,19,19
Republic of Indonesia,19,19,19
Socialist Republic of Vietnam,17,17,17
Republic of India,16,16,16
Republic of Yemen,13,13,13
People's Republic of Bangladesh,12,12,12
Nepal,12,12,12
Kingdom of Morocco,12,12,12
Republic of Mozambique,11,11,11


### Answer: 

Top 10 countries with most projects are: 
1. People's Republic of China
2. Republic of Indonesia
3. Socialist Republic of Vietnam
4. Republic of India
5. Republic of Yemen
6. People's Republic of Bangladesh
7. Nepal
8. Kingdom of Morocco
9. Republic of Mozambique
10. Africa

### Question 2

2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [9]:
# # Make dataframe out of the json file for debugging purposes
json_df = pd.read_json('data/world_bank_projects.json')

# # Subset dataframe of our interest 
test_df = json_df[['_id','countryname','theme_namecode','mjtheme','mjtheme_namecode','mjthemecode']]
test_df.head()

# # Data Exploration
# cols = json_df.columns
# for i in cols:
#     print(i)

# Column with values in list
# json_df[['mjtheme']]

# # # Main Question
# # flatten out json files using json raw data
with open('data/world_bank_projects.json') as file: 
    data = json.load(file)

# my code takes keyword arguments / example above takes positional arguments 
flat_json = json_normalize(data, record_path='mjtheme_namecode', meta=['countryname'])

# # Data Exploration for flat_json
# len(flat_json['code'].unique())  # 11 unique values (for mjtheme code)
# flat_json[flat_json['code']=='1'].shape  # number of rows for which the 'code' column is '1'

# # Return the top 10 codes 
value_counts = flat_json['code'].value_counts()
value_counts


11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
3      15
Name: code, dtype: int64

### Answer: 

Top 10 major project themes by codes are 

1. 11
2. 10
3. 8
4. 2
5. 6
6. 4
7. 7
8. 5 
9. 9
10. 1


* Major project themes by name to follow on the next exercise*


### Question 3

(Partically Combined with Question 2)
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [10]:

import numpy as np

# # Main Question
# # flatten out json files using json raw data
with open('data/world_bank_projects.json') as file: 
    data = json.load(file)

flat_json = json_normalize(data, record_path='mjtheme_namecode', meta=['countryname'])

# To fill in the missing values of the 'name' column, replace the empty strings "" with np.nan
# values and fill the entries in (using .fillna() method) after sorting the columns
sorted_df = flat_json.sort_values(by=['code','name'],ascending=[True, True])
sorted_df.name.replace('', np.nan, inplace=True)
# check that cell entry is nan: 
# print(sorted_df.iloc[0]['name'])

## fill in name column using 'backfill' method -> thus must sort 'name' column in ascending order
## so that the nan values appear first 
sorted_df = sorted_df.fillna(method='bfill')
sorted_df


# # make a mapping dictionary using 'code' column as key and 'name' column as value: 
# method: by zipping two lists together (from the dataframe columns) to avoid using for loops! 
mapping_dict = dict(zip(sorted_df.code, sorted_df.name))

# # Return the top 10 codes with code + theme + counts 
value_counts = flat_json['code'].value_counts()
df_value_counts = pd.DataFrame(value_counts).reset_index()
df_value_counts.columns = ['theme_code','counts']
df_value_counts['theme'] = df_value_counts['theme_code'].map(mapping_dict)
# rearranging the columns 
df_final = df_value_counts[['theme_code','theme','counts']]
df_final



,theme_code,theme,counts
0,11,Environment and natural resources management,250
1,10,Rural development,216
2,8,Human development,210
3,2,Public sector governance,199
4,6,Social protection and risk management,168
5,4,Financial and private sector development,146
6,7,Social dev/gender/inclusion,130
7,5,Trade and integration,77
8,9,Urban development,50
9,1,Economic management,38


### Answer: 

Top 10 themes are 
1. Environment and natural resources management - 250 times
2. Rural development - 216 times
3. Human development - 210 times
4. Public sector governance	- 199 times
5. Social protection and risk management - 168 times
6. Financial and private sector development - 146 times
7. Social dev/gender/inclusion	- 130 times
8. Trade and integration - 77 times
9. Urban development - 50 times
10. Economic management	- 38 times
